# 🎮 Анализатор истории покупок Steam с расчетом инфляции

## 📚 Навигация по ноутбуку

### Быстрый старт (для нетерпеливых 😄):
1. **Подготовка** → Загрузите CSV-файл с историей покупок
2. **Основная обработка** → Запустите всё по порядку
3. **Результаты** → Смотрите `print_results(df_parsed)` в конце

---

## 📥 Как получить CSV-файл с историей покупок Steam

### ✅ Самый простой способ (рекомендуется):

Используйте расширение Chrome для автоматического экспорта истории:

1. Установите расширение для Chrome

    🔗 [Steam Purchase History Exporter (CSV)](https://chromewebstore.google.com/detail/bgbjngfnigoocbhciecckgfjbkkleghp)

2. 🌐 Откройте [Историю покупок Steam](https://store.steampowered.com/account/history/)

3. 📊 Нажмите кнопку **"Export"** в правом верхнем углу таблицы

4. ✨ CSV-файл автоматически загрузится в папку загрузок вашего браузера


### 📖 Чуть подробнее про расширение:
- **Автор:** [dogagcmnr на GitHub](https://github.com/dogagcmnr)
- **Репозиторий:** [steamexporter](https://github.com/dogagcmnr/steamexporter)
- **Что оно делает:** Парсит вашу историю покупок прямо из Steam и экспортирует в CSV

---

## 📝 Если хотите экспортировать вручную:

Если расширение не сработает, используйте этот способ:

1. Откройте [Историю покупок Steam](https://store.steampowered.com/account/history/)
2. Скопируйте всё содержимое таблицы
3. Откройте Excel или другой редактор таблиц
4. Вставьте данные
5. Отредактируйте данные так, чтобы они были в формате 

    ```
    "Date","Items","Type","Total","Wallet Change","Wallet Balance"
    ```

6. Сохраните как CSV файл с именем **`steam_purchase_history.csv`**

---

## 🎯 Что дальше:

1. **Положите файл** в ту же папку, где находится этот ноутбук
2. **Запустите ячейки** сверху вниз — программа автоматически найдёт файл

---

## ⚠️ Важные моменты:

- ✅ Файл должен быть в формате **CSV** (не Excel)
- ✅ Название файла должно быть точно **`steam_purchase_history.csv`**
- ✅ Файл должен быть в **одной папке** с этим ноутбуком
- ⚠️ Если ошибка при загрузке — проверьте кодировку файла (должна быть UTF-8)

---

## 🚀 Полный список секций программы:

| № | Секция | Описание |
|---|--------|---------|
| 1️⃣ | **Импорт библиотек** | Загружаем необходимые инструменты |
| 2️⃣ | **Загрузка данных** | Загружаем CSV-файл с историей Steam |
| 3️⃣ | **Парсинг и обработка** | Очищаем и структурируем данные |
| 4️⃣ | **Анализ игр** | Ищем повторные и дорогие покупки |
| 5️⃣ | **Анализ типов** | Разбираемся с типами транзакций |
| 6️⃣ | **Инфляция** | Пересчитываем цены с учетом инфляции |
| 7️⃣ | **Итоговый отчет** | Смотрим красивую статистику |

---

## 💡 Советы при использовании:

- 🔍 Используйте `Ctrl+F` (или `Cmd+F` на Mac) для поиска по названиям функций
- ⏸️ Запускайте ячейки последовательно (сверху вниз), не пропускайте
- 📊 Если ошибка — прочитайте сообщение об ошибке и проверьте формат CSV
- 💾 Результаты не сохраняются автоматически — скопируйте интересующие вас цифры

---

## 1️⃣ Импорт библиотек

**Что здесь:** Загружаем все необходимые инструменты (библиотеки) для работы

**Какие библиотеки:**
- `pandas` - для работы с таблицами данных (DataFrame)
- `requests` - для загрузки данных из интернета
- `re` - для работы с текстом (регулярные выражения)
- `pycbrf` - курсы валют от Центрального банка России
- `BeautifulSoup` - парсинг веб-страниц (для инфляции)

**Действие:** Просто запустите эту ячейку, она загрузит все библиотеки

---

In [17]:
import requests
import pandas as pd
import re
from pycbrf import ExchangeRates
from bs4 import BeautifulSoup

## 2️⃣ Загрузка истории покупок Steam

### Что здесь происходит:
Загружаем CSV файл с вашей историей покупок в программу


In [ ]:
# Укажите адрес и имя файла с историей покупок Steam
steam_history = 'Maverick29rus_steam_purchase_history.csv'


In [19]:
# Загружаем CSV файл в pandas DataFrame
# parse_dates=[0] означает, что первый столбец (дата) будет преобразован в формат datetime
df = pd.read_csv(steam_history, parse_dates=[0])

In [20]:
# Показываем первые 10 строк данных для проверки корректности загрузки
df.head(10)

,Date,Items,Type,Total,Wallet Change,Wallet Balance
0,2025-03-05,Currency Conversion to RUB (Russian Federation),Conversion / Wallet,NaN,NaN,"323,40 руб."
1,2025-03-05,Purchased $1.01 USD Wallet Credit,Purchase / Kiosk,$1.01 USD,NaN,NaN
2,2023-07-17,Steam Community Market,Market Transaction / Wallet,"1,01 руб. / Credit","+1,01 руб.","233,16 руб."
3,2023-07-15,Steam Community Market,7 Market Transactions / Wallet,"16,41 руб.","-16,41 руб.","232,15 руб."
4,2023-07-01,Steam Community Market,3 Market Transactions / Wallet,"20,21 руб. / Credit","+20,21 руб.","248,56 руб."
5,2023-06-30,Steam Community Market,39 Market Transactions / Wallet,"60,43 руб. / Credit","+60,43 руб.","228,35 руб."
6,2023-02-19,Steam Community Market,Market Transaction / Wallet,"2,07 руб. / Credit","+2,07 руб.","167,92 руб."
7,2023-02-04,Steam Community Market,Market Transaction / Wallet,"2,12 руб. / Credit","+2,12 руб.","165,85 руб."
8,2023-02-01,Steam Community Market,Market Transaction / Wallet,"1,46 руб. / Credit","+1,46 руб.","163,73 руб."
9,2023-01-30,Steam Community Market,Market Transaction / Wallet,"1,99 руб. / Credit","+1,99 руб.","162,27 руб."


In [21]:
# Показываем информацию о DataFrame (типы данных, пропущенные значения и т.д.)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            617 non-null    datetime64[ns]
 1   Items           617 non-null    object        
 2   Type            617 non-null    object        
 3   Total           615 non-null    object        
 4   Wallet Change   560 non-null    object        
 5   Wallet Balance  562 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 29.1+ KB


## 3️⃣ Парсинг и обработка данных

### Что здесь происходит:
Преобразуем сырые данные из CSV в удобный формат для анализа

### 🔄 Процесс обработки:
1. **Очищаем названия** - берём текст, удаляем лишние символы
2. **Извлекаем товары** - выделяем названия игр и имена получателей подарков
3. **Конвертируем валюты** - если покупка была в $, €, £ и т.д., переводим в рубли
4. **Определяем тип** - Purchase (покупка), Gift (подарок), Market (маркет) и т.д.

### 🔧 Технические детали:
- Используется ЦБ РФ API для исторических курсов обмена
- Валюты автоматически определяются по символам ($, €, ₽ и т.д.)
- Данные парсятся строка за строкой для точности

### ⏭️ После запуска:
- Программа создаст новый DataFrame `df_parsed` с очищенными данными
- Можно просмотреть результаты в таблице ниже

---

In [22]:
# ============================================================================
# ОСНОВНЫЕ КОНСТАНТЫ И ФУНКЦИИ ДЛЯ ОБРАБОТКИ ДАННЫХ
# ============================================================================

# ----------------------------
# Константы (текстовые маркеры)
# ----------------------------
# Эти переменные используются для идентификации типов транзакций в Steam

WALLET_CREDIT = "Wallet Credit"  # Пополнение кошелька Steam
REFUND = "Refund"  # Возврат средств
GIFT_INVENTORY = "Stored in gift inventory"  # Подарок в инвентаре
GIFT_SENT_PREFIX = "Gift sent to "  # Подарок отправлен кому-то

# Словарь для преобразования символов валют в коды (ISO 4217)
CURRENCY_SYMBOLS = {
    "$": "USD",      # Доллар США
    "€": "EUR",      # Евро
    "£": "GBP",      # Британский фунт
    "₽": "RUB",      # Российский рубль
    "₴": "UAH",      # Украинская гривня
    "₸": "KZT",      # Казахский тенге
}


# ----------------------------
# Функции для работы с валютами
# ----------------------------

def get_currency_rate(date, currency: str) -> float:
    """
    Получает официальный курс обмена валюты на конкретную дату.
    
    Использует API ЦБ РФ (Центральный банк России) для получения
    исторических курсов обмена.
    
    Args:
        date: Дата, на которую нужен курс (тип datetime)
        currency: Трёхбуквенный код валюты (например 'USD', 'EUR')
    
    Returns:
        float: Курс обмена валюты к рублю на указанную дату
        
    Example:
        >>> get_currency_rate(pd.Timestamp('2024-01-15'), 'USD')
        90.5
    """
    rates = ExchangeRates(date, locale_en=True)
    return float(rates[currency].rate)  # type: ignore


# ----------------------------
# Функции для векторной обработки (работают со всеми строками сразу)
# ----------------------------

def normalize_type(series: pd.Series) -> pd.Series:
    """
    Нормализует и очищает текст типа транзакции.
    
    Удаляет:
    - Переносы строк
    - Информацию после слеша (например, "Purchase / DLC" → "Purchase")
    - Номера в начале (например, "1 Gift Purchase" → "Gift Purchase")
    - Лишние буквы 's' (например, "Transactions" → "Transaction")
    
    Args:
        series: pandas Series со строками типов транзакций
    
    Returns:
        pandas Series с очищенными типами
    """
    return (
        series.fillna("")
        .str.replace("\n", "", regex=False)
        .str.replace(r"\s/.*", "", regex=True)
        .str.replace(r"^\d+\s+", "", regex=True)
        .str.replace("Transactions", "Transaction", regex=False)
    )


# ----------------------------
# Функции для парсинга полей Items (товары и получатели подарков)
# ----------------------------

def parse_items(items: str) -> tuple[str, str]:
    """
    Разбирает поле Items (товары) из истории Steam.
    
    Выделяет:
    1. Названия купленных игр/товаров
    2. Имена получателей подарков (если покупка была подарком)
    
    Обрабатывает следующие форматы:
    - "Game Name" (обычная покупка)
    - "Game 1 / Game 2" (несколько игр)
    - "Game Name / Gift sent to Vasya" (подарок)
    - "Game Name / Stored in gift inventory" (подарок в инвентаре)
    - "Wallet Credit" (пополнение кошелька)
    
    Args:
        items: Строка из поля Items CSV файла
    
    Returns:
        tuple[str, str]: Кортеж (игры, получатели)
            - игры: перечисленные через ", " названия товаров
            - получатели: перечисленные через ", " имена людей или "Stored in gift inventory"
            
    Example:
        >>> parse_items("Portal 2 / Gift sent to Alex")
        ('Portal 2', 'Alex')
        >>> parse_items("Dota 2 / Half-Life 3")
        ('Dota 2, Half-Life 3', '')
    """
    if not items or pd.isna(items):
        return "", ""

    if WALLET_CREDIT in items:
        parts = [WALLET_CREDIT]
    else:
        parts = items.split(" / ")

    games = []
    recipients = []

    index = 0
    while index < len(parts):
        part = parts[index].strip()
        index += 1

        if not part:
            continue

        games.append(part)

        if index >= len(parts):
            continue

        next_part = parts[index]

        if next_part == GIFT_INVENTORY:
            recipients.append(GIFT_INVENTORY)
            index += 2
        elif next_part.startswith(GIFT_SENT_PREFIX):
            recipient = next_part.removeprefix(GIFT_SENT_PREFIX).strip()
            recipients.append(recipient)
            index += 2

    return ", ".join(games), ", ".join(recipients)


# ----------------------------
# Функции для парсинга суммы и конвертирования валют
# ----------------------------

def parse_balance(balance: str, date) -> float:
    """
    Парсит сумму денег, определяет её валюту и конвертирует в рубли.
    
    Процесс обработки:
    1. Извлекает валюту из строки (например, "$", "€", "₽")
    2. Удаляет символ валюты и вспомогательные символы
    3. Преобразует строку в число (заменяет "," на ".")
    4. Если валюта не рубль - конвертирует в рубли по курсу ЦБ РФ на дату
    5. Округляет результат до 2 знаков после запятой
    
    Args:
        balance: Строка с суммой и символом валюты (например "$19.99", "€4,99", "₽499")
        date: Дата транзакции для получения правильного курса обмена
    
    Returns:
        float: Сумма в рублях, округленная до 2 знаков
        
    Example:
        >>> parse_balance("$9.99", pd.Timestamp('2024-01-15'))
        900.45  # если курс доллара 90.15
        >>> parse_balance("₽999", pd.Timestamp('2024-01-15'))
        999.0
    """
    if not balance or pd.isna(balance):
        return 0.0

    raw = balance.strip()
    currency = None

    # Определяем валюту по символу
    for symbol, code in CURRENCY_SYMBOLS.items():
        if symbol in raw:
            raw = raw.replace(symbol, "")
            currency = code
            break

    # Удаляем всё кроме цифр, точек и тире
    raw = re.sub(r"[^\d,-]", "", raw)
    raw = raw.replace(",", ".")  # Некоторые локали используют "," как разделитель

    value = float(raw)

    # Если это не рубли - конвертируем в рубли по курсу ЦБ РФ
    if currency:
        value *= get_currency_rate(date, currency)

    return round(value, 2)


# ----------------------------
# Основная функция обработки DataFrame
# ----------------------------

def parse_transactions(df: pd.DataFrame) -> pd.DataFrame:
    """
    Полная обработка DataFrame с историей покупок Steam.
    
    Обрабатывает сырые данные из CSV и выделяет важную информацию:
    - Дата транзакции
    - Тип (Purchase, Gift Purchase, Market Transaction, In-Game Purchase, Refund и т.д.)
    - Названия товаров/игр
    - Имена получателей подарков (если применимо)
    - Сумма в рублях
    
    Особенности:
    - Для операций на маркете используется поле "Wallet Change" вместо "Total"
    - Все суммы конвертируются в рубли с использованием исторических курсов ЦБ РФ
    - Обработка оптимизирована без использования apply() для высокой производительности
    
    Args:
        df: pandas DataFrame с сырыми данными из CSV (должен иметь колонки:
            Date, Type, Items, Total, Wallet Change)
    
    Returns:
        pandas DataFrame с обработанными данными, содержащий колонки:
        - date: Дата транзакции
        - type: Нормализованный тип транзакции
        - games: Названия товаров (игр), разделённые запятыми
        - recipients: Имена получателей подарков, разделённые запятыми
        - total: Сумма в рублях
    """
    result = pd.DataFrame(index=df.index)

    result["date"] = df["Date"]
    result["type"] = normalize_type(df["Type"])

    # Для операций на маркете используется "Wallet Change", для остальных "Total"
    amount_source = df["Total"].where(
        result["type"] != "Market Transaction",
        df["Wallet Change"],
    )

    games = []
    recipients = []
    totals = []

    # Обработаем каждую строку один раз
    for items, amount, date in zip(
        df["Items"],
        amount_source,
        df["Date"],
    ):
        game_list, recipient_list = parse_items(items)

        games.append(game_list)
        recipients.append(recipient_list)
        totals.append(parse_balance(amount, date))

    result["games"] = games
    result["recipients"] = recipients
    result["total"] = totals

    return result

In [23]:
# Применяем функцию обработки к данным
# Результат - новый DataFrame с очищенными и структурированными данными
df_parsed = parse_transactions(df)

In [24]:
# Показываем результаты обработки
# Теперь данные структурированы и готовы к анализу
df_parsed

,date,type,games,recipients,total
0,2025-03-05,Conversion,Currency Conversion to RUB (Russian Federation),,0.00
1,2025-03-05,Purchase,Wallet Credit,,9013.72
2,2023-07-17,Market Transaction,Steam Community Market,,1.01
3,2023-07-15,Market Transaction,Steam Community Market,,-16.41
4,2023-07-01,Market Transaction,Steam Community Market,,20.21
...,...,...,...,...,...
612,2012-07-14,Purchase,"Tomb Raider: Underworld, Tomb Raider: Anniversary",,160.00
613,2012-07-13,Purchase,Back to the Future: The Game,,111.00
614,2012-05-27,Purchase,Max Payne 3 Pre-order 2(RU),,599.00
615,2011-12-31,Gift Purchase,Magicka Complete Pack (Nov 2011),,219.00


In [25]:
# Информация о структуре обработанных данных
df_parsed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617 entries, 0 to 616
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        617 non-null    datetime64[ns]
 1   type        617 non-null    object        
 2   games       617 non-null    object        
 3   recipients  617 non-null    object        
 4   total       617 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 24.2+ KB


## 4️⃣ Анализ игр и фильтрация данных

### Что здесь происходит:
Смотрим, какие игры куплены, ищем дубликаты и удаляем ненужные записи

### 🎮 Проверяем:
1. **Повторные покупки** - показываем игры, купленные больше одного раза
2. **Пополнение кошелька** - удаляем (это не покупка товара)
3. **Типы транзакций** - видим, какие покупки есть в истории
4. **Конвертации** - удаляем обмены валют (не относятся к покупкам)
5. **Возвраты** - находим возвращенные товары и связанные с ними покупки

### ❓ Почему это нужно:
- Пополнение кошелька не является покупкой игры
- Конвертации валют - технические операции
- Некорректные типы будут исказить статистику

### 📋 Что делать с результатами:
- Если видите строки с ошибками - можно их удалить вручную
- Возвраты можно исключить из анализа (раскомментировать код)
- Остальные данные готовы к дальнейшему анализу

---

In [26]:
# Показываем игры, которые были куплены несколько раз (количество > 1)
# Это помогает найти дубликаты или повторные покупки
df_parsed.games.value_counts()[lambda x: x > 1]

games
Steam Community Market                                          329
Wallet Credit                                                    42
The Witcher: Enhanced Edition                                     3
Currency Conversion to RUB (Russian Federation)                   2
STAR WARS™ Knights of the Old Republic™                           2
Left 4 Dead Bundle                                                2
GTA Complete Pack (ROW)                                           2
Batman Arkham City GOTY                                           2
Serious Sam II                                                    2
Half-Life 2                                                       2
Saints Row Ultimate Franchise Pack                                2
World of Goo                                                      2
STAR WARS™ Knights of the Old Republic™ II - The Sith Lords™      2
The Witcher 2 (RU)                                                2
Magicka Complete Pack (Nov 2011)          

In [27]:
# Удаляем записи о пополнении кошелька Steam из основного анализа
# Это не являются покупками игр, а просто добавлением средств
df_parsed = df_parsed[df_parsed.games != 'Wallet Credit']

## Анализ типов транзакций

Проверяем, какие типы транзакций есть в истории, и удаляем те, которые не нужны для анализа

In [28]:
# Показываем статистику по типам транзакций
# Это поможет понять, какие типы есть в данных
df_parsed['type'].value_counts()

type
Market Transaction    329
Purchase              182
Gift Purchase          54
In-Game Purchase        5
Refund                  3
Conversion              2
Name: count, dtype: int64

In [29]:
# Удаляем "Conversion" (обмены валют) - это технические операции, а не реальные покупки
df_parsed = df_parsed[df_parsed.type != 'Conversion']

In [30]:
# Показываем все возвраты в отдельном представлении
df_parsed[df_parsed.type == 'Refund']

,date,type,games,recipients,total
185,2016-12-25,Refund,"Wolfenstein: The Old Blood ROW, Refund",,263.0
187,2016-12-25,Refund,"Stories: Path of Destinies, Refund",,174.0
324,2015-06-22,Refund,Refund,,685.0


In [47]:
def find_refund(df):
    """
    Находит все операции (покупки и возвраты), связанные с возвращенными товарами.
    
    Как это работает:
    1. Извлекает названия всех игр, по которым был сделан возврат
    2. Ищет все записи в DataFrame, которые содержат эти названия
    3. Возвращает DataFrame со всеми связанными операциями (покупка + возврат)
    
    Это помогает увидеть полную историю возвращенных товаров.
    
    Args:
        df: DataFrame с обработанными транзакциями
    
    Returns:
        DataFrame со всеми операциями, связанными с возвратами
    """
    # Получаем названия игр, по которым был возврат
    refund_names = []
    for game in df[df.type == 'Refund'].games:
        refund_names.extend(game.split(', '))

    # Функция для проверки, содержит ли строка из DataFrame любую из строк в списке
    def contains_any_search_string(s, search_strings):
        return any(substring in s for substring in search_strings)

    # Фильтруем DataFrame, ищем все записи с названиями возвращенных игр
    filtered_df = df[
        df.games.apply(lambda x: contains_any_search_string(x, refund_names))
    ]

    return filtered_df

In [48]:
# Ищем все операции, связанные с возвратами (покупка + возврат)
refund = find_refund(df_parsed)
refund

,date,type,games,recipients,total,total_inf
183,2016-12-25,Purchase,"Stories: Path of Destinies, Wolfenstein: The O...",,1336.99,2337.89
185,2016-12-25,Refund,"Wolfenstein: The Old Blood ROW, Refund",,263.00,459.89
187,2016-12-25,Refund,"Stories: Path of Destinies, Refund",,174.00,304.26
189,2016-12-25,Purchase,"Stories: Path of Destinies, Wolfenstein: The O...",,437.00,764.15
324,2015-06-22,Refund,Refund,,685.00,1310.51


In [33]:
# [ОПЦИОНАЛЬНО] Если вы хотите исключить возвраты из анализа, выберите индексы 
# возвращенных товаров из выходных данных выше и раскомментируйте код ниже.
# Например, если нужно удалить возвраты с индексами 185, 187, 189, 324:
# refunds_to_drop = [185, 187, 189, 324]
# df_parsed.drop(refunds_to_drop, inplace=True)

## 5️⃣ Проверка и очистка сумм

### Что здесь происходит:
Проверяем, что все суммы правильные, и удаляем ошибочные записи

### 🔍 Ищем:
- Записи с суммой = 0 (обычно это ошибки при парсинге)
- Некорректно распознанные цены

### 💪 Почему это важно:
- Нулевые суммы исказят среднюю стоимость и общую сумму
- Могут быть ошибки парсинга при необычном формате текста
- Также удаляем все нулевые значения из дальнейшего анализа

---

In [34]:
# Показываем все записи с нулевой суммой (обычно это ошибки при парсинге)
df_parsed[df_parsed.total == 0]

,date,type,games,recipients,total


In [35]:
# Удаляем записи с нулевой суммой
df_parsed = df_parsed[df_parsed.total != 0]

## 6️⃣ Расчет инфляции - самая интересная часть! 📈

### 🤔 Что такое инфляция в этом контексте?
Инфляция - это когда со временем на ту же сумму денег можно купить всё меньше товаров.

**Пример в цифрах:**
- Вы купили игру в 2020 году за **100 рублей**
- Инфляция за 4 года была 50%
- Эквивалентная стоимость в 2024 = **150 рублей**
- Это значит, что чтобы иметь такую же покупательную способность, нужно потратить на 50% больше

### 📊 Как это работает в нашей программе:

1. **Загружаем данные об инфляции** с официального сайта (уровень-инфляции.рф)
2. **Используем ЦБ РФ API** для курсов валют на конкретные даты
3. **Для каждой покупки считаем:** от даты покупки до сегодня - какая была инфляция
4. **Пересчитываем цены:** умножаем исходную цену на коэффициент инфляции

### 📉 Интерпретация результатов:
- `total` - сколько вы потратили **в момент покупки**
- `total_inf` - эквивалентная стоимость **в деньгах сегодня**

**Пример:**
```
Purchase 1: total = 500₽ (2020)  →  total_inf = 750₽
            Это значит, 500₽ в 2020 = 750₽ в 2024
```

### ⏭️ Что будет дальше:
- Программа автоматически загрузит данные об инфляции (может занять пару секунд)
- Добавит новую колонку `total_inf` со всеми расчитанными ценами
- Эти данные будут использованы в финальный отчёте

### ⚠️ Технические моменты:
- Если загрузка не работает - проверьте интернет соединение
- Данные берутся с официального источника российской инфляции
- Для нерублевых покупок сначала конвертируем в рубли, потом считаем инфляцию

---

In [36]:
def get_inflation_data():
    """
    Загружает и парсит данные об инфляции рубля с сайта уровень-инфляции.рф
    
    Процесс работы:
    1. Отправляет HTTP запрос на сайт с таблицей инфляции
    2. Парсит HTML таблицу с помощью BeautifulSoup
    3. Извлекает данные об инфляции по месяцам в процентах
    4. Возвращает pandas Series с индексом в виде дат
    
    **Что такое инфляция:**
    - Это процент увеличения средних цен на товары за месяц
    - Например, если инфляция за январь 1.5%, значит все товары в среднем подорожали на 1.5%
    
    Returns:
        pandas.Series: Где индекс - первый день месяца (datetime),
                       значение - инфляция за этот месяц в процентах (float)
                       
    Example output:
        2020-01-01    0.3
        2020-02-01    0.4
        2020-03-01    0.6
        ...
        Dtype: float64
        
    Raises:
        Exception: Если невозможно получить данные с сайта
    """
    url = "https://xn----ctbjnaatncev9av3a8f8b.xn--p1ai/таблицы-инфляции"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }

    def parse_table(rows):
        """
        Парсит HTML таблицу с данными об инфляции.
        
        Таблица имеет следующую структуру:
        - Первый столбец: год (2020, 2021, 2022 и т.д.)
        - Следующие 12 столбцов: инфляция по месяцам (январь-декабрь)
        
        Args:
            rows: Список <tr> элементов из HTML таблицы
        
        Returns:
            dict: Ключ - строка в формате 'YYYY-MM', значение - инфляция в %
        """
        data = {}

        for row in rows[1:]:  # Пропускаем заголовок
            cells = row.find_all(['td', 'th'])

            year = int(cells[0].get_text(strip=True))

            for month in range(1, 13):
                value_text = cells[month].get_text(strip=True)

                if value_text:
                    inflation = float(value_text)
                    data[f'{year}-{month}'] = inflation

        return data

    try:
        # Загружаем страницу
        response = requests.get(url, headers=headers, timeout=10)
        response.encoding = 'utf-8'

        if response.status_code != 200:
            print(f"Ошибка HTTP: {response.status_code}")
            return None

        # Парсим HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Ищем и обрабатываем таблицы
        tables = soup.find_all('table')
        rows = tables[1].find_all('tr')  # Берем вторую таблицу
        inflation_data = parse_table(rows)
        
        # Преобразуем в pandas Series с datetime индексом
        df_inflation = pd.Series(inflation_data, dtype='float64')
        df_inflation.index = pd.to_datetime(df_inflation.index)

        return df_inflation.sort_index()

    except Exception as e:
        print(f"Ошибка при получении данных: {str(e)}")
        return None

In [37]:
# Загружаем данные об инфляции с сайта
# Это может занять несколько секунд при первом запуске
inflation_data = get_inflation_data()
inflation_data

1991-01-01     6.20
1991-02-01     4.80
1991-03-01     6.30
1991-04-01    63.50
1991-05-01     3.00
              ...  
2025-08-01    -0.40
2025-09-01     0.34
2025-10-01     0.50
2025-11-01     0.42
2025-12-01     0.32
Length: 420, dtype: float64

In [ ]:
def calculate_inflation_between_dates(inflation_data, start_date_str, end_date_str):
    """
    Вычисляет накопленную инфляцию между двумя датами.
    
    **Что это делает:**
    Берет ежемесячные данные об инфляции и считает, на сколько процентов
    подорожали товары с начальной даты до конечной.
    
    **Пример:**
    - start_date: '2023-01-15' (15 января 2023)
    - end_date: '2024-01-15' (15 января 2024)
    - Если за этот период инфляция была 15%, функция вернет 1.15
    - Это значит, что 100₽ год назад стоили бы 115₽ сегодня
    
    **Формула:**
    inflation_rate = (1 + инфляция_месяц_1) * (1 + инфляция_месяц_2) * ... * (1 + инфляция_месяц_N)
    
    Args:
        inflation_data: pandas.Series с данными об инфляции 
                        (индекс: datetime, значение: инфляция)
        start_date_str: Начальная дата в формате 'YYYY-MM' или 'YYYY-MM-DD'
                        Пример: '2023-06-15' или '2023-06'
        end_date_str: Конечная дата в формате 'YYYY-MM' или 'YYYY-MM-DD'
                      Пример: '2024-01-01' или '2024-01'
    
    Returns:
        float: Коэффициент накопленной инфляции
               1.0 = нет инфляции
               1.15 = 15% инфляция (требуется на 15% больше денег для того же товара)
               0.95 = дефляция (-5%, товары подешевели)
    
    Raises:
        ValueError: Если даты некорректные или конец раньше начала
        
    Example:
        >>> rate = calculate_inflation_between_dates(inflation_data, '2023-01', '2024-01')
        >>> rate
        1.1526  # За год инфляция 15.26%
    """

    # Преобразуем строки дат в объекты datetime
    def parse_date(date_str):
        try:
            # Берем первый день месяца (всегда используем 1-е число)
            return pd.to_datetime(date_str).normalize() + pd.offsets.MonthBegin(-1)
        except Exception as e:
            raise ValueError(f"Ошибка при парсинге даты {date_str}: {str(e)}")

    start_date = parse_date(start_date_str)
    end_date = parse_date(end_date_str)

    # Рассчитываем накопленную инфляцию
    if end_date < start_date:
        raise ValueError("Конечная дата должна быть позже начальной")
    if end_date == start_date:
        return 1
    else:
        # Берем инфляцию с start_date по end_date, преобразуем в коэффициент
        # и считаем произведение (накопленный эффект)
        cumulative_inflation = (
            inflation_data.loc[start_date:end_date] / 100 + 1
        ).cumprod()
        return cumulative_inflation.iloc[-1]

In [39]:
def calculate_price(row, col):
    """
    Вспомогательная функция для применения к каждой строке DataFrame.
    
    Берет цену из указанной колонки и пересчитывает её с учетом инфляции
    от даты покупки до сегодняшнего дня.
    
    Args:
        row: Одна строка DataFrame (серия данных)
        col: Название колонки с ценой ('total')
    
    Returns:
        float: Цена с учетом инфляции, округленная до 2 знаков
    """
    # Получаем коэффициент инфляции от даты покупки до сегодня
    inflation_rate = calculate_inflation_between_dates(
        inflation_data, row['date'], pd.Timestamp.now()
    )
    # Применяем коэффициент инфляции к цене
    price = inflation_rate * row[col]
    return round(price, 2)

In [40]:
# Добавляем новую колонку 'total_inf' с ценами, пересчитанными с учетом инфляции
# Это может занять некоторое время, так как для каждой покупки нужно получить курс на дату
df_parsed['total_inf'] = df_parsed.apply(calculate_price, axis=1, col=('total'))

In [41]:
# Показываем итоговый DataFrame с добавленной колонкой инфляции
df_parsed

,date,type,games,recipients,total,total_inf
2,2023-07-17,Market Transaction,Steam Community Market,,1.01,1.22
3,2023-07-15,Market Transaction,Steam Community Market,,-16.41,-19.84
4,2023-07-01,Market Transaction,Steam Community Market,,20.21,24.52
5,2023-06-30,Market Transaction,Steam Community Market,,60.43,73.32
6,2023-02-19,Market Transaction,Steam Community Market,,2.07,2.55
...,...,...,...,...,...,...
612,2012-07-14,Purchase,"Tomb Raider: Underworld, Tomb Raider: Anniversary",,160.00,405.85
613,2012-07-13,Purchase,Back to the Future: The Game,,111.00,281.56
614,2012-05-27,Purchase,Max Payne 3 Pre-order 2(RU),,599.00,1540.89
615,2011-12-31,Gift Purchase,Magicka Complete Pack (Nov 2011),,219.00,575.86


In [42]:
# Информация о финальном DataFrame
df_parsed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 573 entries, 2 to 616
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        573 non-null    datetime64[ns]
 1   type        573 non-null    object        
 2   games       573 non-null    object        
 3   recipients  573 non-null    object        
 4   total       573 non-null    float64       
 5   total_inf   573 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 31.3+ KB


In [43]:
# Итоговая статистика по типам транзакций в очищенном DataFrame
df_parsed.type.value_counts()

type
Market Transaction    329
Purchase              182
Gift Purchase          54
In-Game Purchase        5
Refund                  3
Name: count, dtype: int64

In [44]:
# Показываем самые дорогие транзакции (исключая маркет) отсортированные по цене с инфляцией
# Market Transaction исключены, так как это обычно мелкие операции на торговой площадке
df_parsed[df_parsed.type != 'Market Transaction'].sort_values(
    by=['total_inf'], ascending=False
)

,date,type,games,recipients,total,total_inf
144,2017-11-23,Purchase,"Borderlands: The Pre-Sequel Season Pass, What ...",,2150.01,3675.84
76,2019-09-14,Purchase,"Vampyr, Vampire: The Masquerade® - Bloodlines™...",,2228.00,3544.16
114,2018-06-25,Purchase,"Observer, Quantum Break, Tales of Symphonia, T...",,1694.09,2833.16
183,2016-12-25,Purchase,"Stories: Path of Destinies, Wolfenstein: The O...",,1336.99,2337.89
326,2015-06-20,Purchase,LEGO&reg; Pirates of the Caribbean The Video G...,,1192.00,2280.48
...,...,...,...,...,...,...
257,2016-01-20,Purchase,God Game : The Odyssey,,5.00,9.18
270,2015-12-22,Purchase,Ampersand,,4.00,7.40
453,2014-12-20,Purchase,Dead Bits,,3.00,6.38
452,2014-12-20,Purchase,GEARCRACK Arena + Soundtrack,,2.00,4.25


## 7️⃣ Итоговый анализ и красивый отчет

### 🎉 Что это:
Финальная функция, которая выводит красивый и читаемый отчет со всеми результатами анализа

### 📊 Что вы увидите в отчёте:

#### 1) **Общая информация о покупках:**
- Сколько игр куплено (не подарков)
- Сколько денег потрачено изначально
- Сколько денег было потрачено в деньгах "сегодня" (с инфляцией)
- Процент инфляции за всё время покупок
- Среднюю стоимость одной игры
- ТОП-10 самых дорогих игр

#### 2) **Игры, купленные несколько раз:**
- Какие игры были куплены больше одного раза

#### 3) **Подарки (если есть):**
- Сколько подарков вы отправили
- Кому и что вы дарили
- Сумму потраченную на подарки

#### 4) **Операции на маркете (если есть):**
- Сколько операций на торговой площадке
- Самые дорогие покупки и продажи

#### 5) **Внутриигровые покупки (если есть):**
- Сколько потрачено на внутриигровые покупки
- В каких играх вы тратили деньги

#### 6) **Возвраты (если есть):**
- Сколько раз вы возвращали товары
- Какие игры вернули

---

In [1]:
def print_results(df):
    """
    ОСНОВНАЯ ФУНКЦИЯ АНАЛИЗА - выводит подробную статистику по всем покупкам.
    
    Эта функция анализирует DataFrame и выводит красиво отформатированный отчет,
    который включает:
    
    1. ОБЩУЮ ИНФОРМАЦИЮ:
       - Количество купленных игр
       - Общую сумму денег, потраченную на игры
       - Эту же сумму, но с учетом инфляции
       - Суммарный процент инфляции за все время
       - Среднюю стоимость одной игры
       - Самые дорогие игры
       - Игры, купленные несколько раз
    
    2. ПОДАРКИ (если они есть):
       - Количество подарков
       - Сумму потраченную на подарки
       - Список получателей подарков
       - Список подаренных игр
    
    3. МАРКЕТ ОПЕРАЦИИ (если они есть):
       - Количество операций
       - Сумму покупок и продаж на маркете
       - Самые дорогие покупки и продажи
    
    4. ВНУТРИИГРОВЫЕ ПОКУПКИ (если они есть):
       - Количество внутриигровых операций
       - Общую сумму внутриигровых покупок
       - Список игр с внутриигровыми покупками
    
    5. ВОЗВРАТЫ (если они есть):
       - Количество возвратов
       - Сумму вернутых денег
       - Список возвращенных товаров
    
    Args:
        df: DataFrame с обработанными транзакциями Steam
    """
    
    # ========== ОБЩАЯ ИНФОРМАЦИЯ ==========
    purchase = df[df.type == 'Purchase']
    type_list = ['Gift Purchase', 'Purchase']
    # Если нужно включить только обычные покупки без подарков, измените на:
    # type_list = ['Purchase']
    
    purchase_with_gifts = df[df.type.isin(type_list)]
    
    # Создаем список всех игр (разбиваем строку с несколькими играми через запятую)
    # Используем list comprehension для компактного и быстрого кода
    games = pd.Series(
        [
            game.strip()
            for item in purchase_with_gifts.games
            if item
            for game in item.split(', ')
        ],
        name='game',
    )
    
    print('Общая информация')
    print('=' * 50)
    print(f'Всего куплено игр: {purchase.shape[0]} шт')
    print(f'Всего стоимость игр: {round(purchase.total.sum(), 2)} р')
    print(f'Всего стоимость игр с учетом инфляции: {round(purchase.total_inf.sum(), 2)} р')
    print(
        f'Суммарная инфляция аккауна: {((purchase.total_inf.sum() / purchase.total.sum()) - 1) * 100:.2f}%'
    )
    print(f'Средняя стоимость игр: {purchase.total.sum() / len(games):.2f} р')
    print(
        f'Средняя стоимость игр с учетом инфляции: {purchase.total_inf.sum() / len(games):.2f} р'
    )
    print('')
    print('Самые дорогие купленные игры:')
    print(
        purchase_with_gifts.loc[:, ['games', 'total_inf']]
        .sort_values(by=['total_inf'], ascending=False)
        .head(10)
    )

    print('')
    print('Игры купленные больше одного раза (включая подарки):')
    print('=' * 50)
    print(
        games.value_counts()[lambda x: x > 1]
        .reset_index()
        .sort_values(['count', 'game'], ascending=[False, True])
        .set_index('game')
    )

    # ========== ПОДАРКИ ==========
    gifts = df[df.type == 'Gift Purchase']
    if not gifts.empty:
        # Создаем список игр, которые были подарены
        gift_games = pd.Series(
            [game.strip() for item in gifts.games if item for game in item.split(', ')],
            name='game',
        )

        # Создаем список получателей подарков
        recipients = pd.Series(
            [
                recipient.strip()
                for item in gifts.recipients
                if item
                for recipient in item.split(', ')
            ]
        )

        print('\n')
        print('Подарки')
        print('=' * 50)
        print(f'Количество подарков: {gifts.shape[0]} шт')
        print(f'Cтоимость подарков: {gifts.total.sum():.2f} р')
        print(f'Cтоимость подарков с учетом инфляции: {gifts.total_inf.sum():.2f} р')
        print(
            f'Суммарная инфляция: {((gifts.total_inf.sum() / gifts.total.sum()) - 1) * 100:.2f}%'
        )
        print('')
        print(f'Список получателей подарков:')
        print(recipients.value_counts())
        print('')
        print(f'Список подарков:')
        print(
            gift_games.value_counts()
            .reset_index()
            .sort_values(['count', 'game'], ascending=[False, True])
            .set_index('game')
        )

    # ========== ОПЕРАЦИИ НА МАРКЕТЕ ==========
    market = df[df.type == 'Market Transaction']
    if not market.empty:
        print('\n')
        print('Операции на маркете')
        print('=' * 50)
        print(f'Количество операций на торговой площадке: {market.shape[0]} шт')
        print(f'Сумма операций на торговой площадке: {round(market.total.sum(), 2)} р')
        print(
            f'Сумма операций на торговой площадке с учетом инфляции: {round(market.total_inf.sum(), 2)} р'
        )
        print(
            f'Суммарная инфляция: {((market.total_inf.sum() / market.total.sum()) - 1) * 100:.2f}%'
        )
        print('')
        print(
            f'Самые дорогие покупки на торговой площадке: \n{market.total.sort_values(ignore_index=True).head()}'
        )
        print('')
        print(
            f'Самые дорогие продажи на торговой площадке: \n{market.total.sort_values(ascending=False, ignore_index=True).head()}'
        )

    # ========== ВНУТРИИГРОВЫЕ ПОКУПКИ ==========
    ingame_purchase = df[df.type == 'In-Game Purchase']
    if not ingame_purchase.empty:
        print('\n')
        print('Внутриигровые покупки')
        print('=' * 50)

        print(f'Количество внутриигровых покупок: {ingame_purchase.shape[0]} шт')
        print(f'Стоимость внутриигровых покупок: {ingame_purchase.total.sum():.2f} р')
        print(
            f'Стоимость внутриигровых покупок с учетом инфляции: {ingame_purchase.total_inf.sum():.2f} р'
        )
        print(
            f'Суммарная инфляция: {((ingame_purchase.total_inf.sum() / ingame_purchase.total.sum()) - 1) * 100:.2f}%'
        )
        print('')
        print(
            f'Список внутриигровых покупок:', *ingame_purchase.games.to_list(), sep='\n'
        )

    # ========== ВОЗВРАТЫ ==========
    refund = df[df.type == 'Refund']
    if not refund.empty:
        print('\n')
        print('Возвраты')
        print('=' * 50)
        print(f'Количество возвратов: {refund.shape[0]} шт')
        print(f'Стоимость возвратов: {refund.total.sum():.2f} р')
        print(f'Стоимость возвратов с учетом инфляции: {refund.total_inf.sum():.2f} р')
        print(
            f'Суммарная инфляция: {((refund.total_inf.sum() / refund.total.sum()) - 1) * 100:.2f}%'
        )
        print('')
        print(f'Список возвратов:', *refund.games.to_list(), sep='\n')

In [2]:
# ============================================================================
# ВЫВОДИМ ИТОГОВЫЙ АНАЛИЗ
# ============================================================================
# Эта функция выводит красиво отформатированный отчет со всей статистикой
# Нажмите Ctrl+Enter (или Cmd+Enter на Mac) для выполнения
print_results(df_parsed)

NameError: name 'df_parsed' is not defined

---

## 🎯 Резюме: что здесь происходит

Эта программа отвечает на вопросы:
- 🤑 **"Сколько я потратил на Steam?"** → Ответ в разделе "Всего стоимость"
- 📈 **"Как изменились цены за время моих покупок?"** → Смотрите "Суммарная инфляция"
- 🎮 **"Какие игры я куплю несколько раз?"** → Раздел "Повторные покупки"
- 🎁 **"Кому я дарил игры?"** → Раздел "Подарки"
- 💸 **"Сколько я потратил на маркет?"** → Раздел "Операции на маркете"

---

## 🔧 Советы по использованию:

### Если что-то не работает:
1. ❓ **Ошибка при загрузке CSV** → проверьте имя файла и формат
2. ❓ **Ошибка валюты** → возможно есть неизвестная символ (добавьте в словарь)
3. ❓ **Ошибка при загрузке инфляции** → проверьте интернет соединение

---

## 📚 Структура данных:

После обработки DataFrame содержит:
```
date        - дата покупки
type        - тип операции (Purchase, Gift Purchase, Market, Refund, и т.д.)
games       - названия купленных товаров
recipients  - имена получателей подарков
total       - сумма в рублях (на дату покупки)
total_inf   - сумма с учетом инфляции (в разговоре на сегодня)
```

---

## 🎓 Если что то не работает или вы заметили ошибку, или в ващей истории покупок есть другие типы операций не учтенных в коде, или есть предложения по улучшению и доработке прошу связаться со мной.

[Telegram](@Maverick29rus)

[Email](Maverick29rus@gmail.com)

---

**Успехов в анализе! Если возникли вопросы - смотрите комментарии в коде 😉**